# Tropical Cyclone Return Period plot for ENSO Phases

This script is designed to calculate and visualize the return periods of tropical cyclones for various cities, based on the historical ENSO (El Niño-Southern Oscillation) phases. The return period is the number of years between events of a certain intensity or size (in this case, wind speed). The script calculates and plots the return periods of tropical cyclones across three ENSO phases: Neutral, El Niño, and La Niña.

### Workflow Overview:

1. **Data Loading**:
   - The script begins by loading an Excel file (`List_of_Cities.xlsx`) that contains the cities' names, latitudes, longitudes, and basins. The list of cities is processed in a loop.

2. **Return Period Calculation**:
   - The `rp_climatology()` function is called for each city and for three different ENSO phases: Neutral, El Niño, and La Niña.
   - The RP data is loaded from CSV files and cleaned by removing outliers (RP values greater than 6000).
   - For each city, the function calculates the mean (`RPmean`) and standard deviation (`RPstd`) of the RP values for the different ENSO phases.
   - The confidence intervals are calculated as `RPb0` (lower bound) and `RPb1` (upper bound), with any negative values being set to NaN.

3. **Return Period Plotting**:
   - The `plot_return_wind()` function creates a plot for each city, showing the return period on the x-axis and the wind speed categories on the y-axis.
   - The return period is displayed using a logarithmic scale to better visualize the data across different ranges.
   - The function generates plots for the Neutral, El Niño, and La Niña phases and saves the resulting figures as PNG images in the specified output directory.

4. **Iterating Over Cities**:
   - For each city in the list, the script processes the data and generates plots. If any city’s data is unavailable or if there is an error, the script skips that city and moves to the next.



This scripts has been developed by Odériz (2025)

## Required Libraries

Before running the script, make sure to install the following libraries:

- `pandas`: For handling and manipulating the cyclone data.
- `numpy`: For mathematical operations and array handling.
- `matplotlib`: For plotting graphs and visualizations.

You can install these libraries using the following command:

```bash
pip install pandas numpy matplotlib 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
def rp_climatology(period,pathOut,city):
    df0=pd.read_csv(pathOut+city+'_'+period+'_0.csv') 
    df1=pd.read_csv(pathOut+city+'_'+period+'_1.csv') 
    df2=pd.read_csv(pathOut+city+'_'+period+'_2.csv') 
    df3=pd.read_csv(pathOut+city+'_'+period+'_3.csv') 
    df4=pd.read_csv(pathOut+city+'_'+period+'_4.csv') 


    df0.loc[df0['RP'] > 6000, 'RP'] = np.nan
    df1.loc[df1['RP'] > 6000, 'RP'] = np.nan
    df2.loc[df2['RP'] > 6000, 'RP'] = np.nan
    df3.loc[df3['RP'] > 6000, 'RP'] = np.nan
    df4.loc[df4['RP'] > 6000, 'RP'] = np.nan


    df=df0.copy()
    df['RP1']=df1['RP'].copy()
    df['RP2']=df2['RP'].copy()
    df['RP3']=df3['RP'].copy()
    df['RP4']=df4['RP'].copy()

    df['RPmean']=df.iloc[:, 2:].mean(axis=1) 
    df['RPstd']=df.iloc[:, 2:].std(axis=1) 

    df['RPb0']=df['RPmean']-df['RPstd']
    df['RPb1']=df['RPmean']+df['RPstd']

    df.loc[df['RPb0'] <0, 'RPb0'] = np.nan
    df.loc[df['RPb1'] <0, 'RPb1'] = np.nan


    return df


def plot_return_wind (df0, df3,city, country,pathOut):
    wind_items = [20, 29,37.6, 43.4,51.1, 61.6]
    cat = ['TS','cat 1','cat 2','cat 3','cat 4','cat 5']

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(3, 3))

    # ax.figure(figsize=(8, 4))  # Set the figure size
    # Customize the plot

    ax.plot(df0['RPmean'], df0['Wind_speed'], color='black', linestyle='-', linewidth=1, markersize=5, label='Neutral ENSO')
    ax.fill_betweenx(df0['Wind_speed'],df0['RPb0'], df0['RPb1'],color='black',alpha=0.1)


    """ ax.plot(df1['RPmean'], df1['Wind_speed'], color='green', linestyle='-', linewidth=1, markersize=5, label='Neutral ENSO')
    ax.fill_betweenx(df1['Wind_speed'],df1['RPb0'], df1['RPb1'],color='green',alpha=0.1) """

    #ax.plot(df2['RPmean'], df2['Wind_speed'], color='red', linestyle='-', linewidth=1, markersize=5, label='EL Niño')
    #ax.fill_betweenx(df2['Wind_speed'],df2['RPb0'], df2['RPb1'],color='red',alpha=0.1)

    ax.plot(df3['RPmean'], df3['Wind_speed'], color='blue', linestyle='-', linewidth=1, markersize=5, label='La Niña')
    ax.fill_betweenx(df3['Wind_speed'],df3['RPb0'], df3['RPb1'],color='blue',alpha=0.1)


    # Set Y-axis custom labels
    ax.set_yticks(wind_items)
    ax.set_yticklabels(cat)
    ax.set_ylim(20, 80)

    # Labels and title
    # Set labels and title
    ax.set_xlabel('Return period (years)')
    ax.set_ylabel('Category')
    #ax.set_title(name+', '+period)

    # Add a legend
    ax.legend(loc='best')

    # Set a logarithmic scale for the x-axis
    ax.set_xlim(0, 1000)    
    ax.set_xscale('log')

    # Grid lines
    ax.grid(True, linestyle='--', alpha=0.7)

    # Create a twin y-axis on the right
    ax2 = ax.twinx()
    ax2.set_yticks(wind_items)
    ax2.set_yticklabels(wind_items)
    ax2.set_ylabel('Wind speed (m/s)')
    ax2.set_ylim(20, 80)
    ax.set_xlim(0, 1000)
    ax.set_title(city+' ('+country+') for 1980-2021')

    plt.savefig(pathOut+city+'_'+country+'.png', format='png', dpi=600,bbox_inches='tight')
    # Show the plot
    plt.show()
    plt.close('all')



    

In [ ]:
# Path to the output folder where the PNG files will be saved
pathOut=r'D:\01.Papers\2024_ENSO_cyclones\05_return_period/'

# Excel file containing the city list (make sure the file is accessible)
folder_name = 'List_of_Cities.xlsx'


period_names=['IB1980-2021LANINA.v1','IB1980-2021ENSO_NEUTRAL.v1','IB1980-2021ELNINO.v1','IB1980-2021']


#==================================================== 
# Load cities                 
#==================================================== 
list_cities=pd.read_excel(pathOut+folder_name,header=0,keep_default_na=False)
latitudes=list_cities['LATITUDE']
longitudes=list_cities['LONGITUDE']
basins=list_cities['BASIN']
names=list_cities['NAME']
capitals=list_cities['CAPITAL CITY']


for i in range(len(capitals)):

    try:
        #df_all=rp_climatology(period_names[3],pathOut,capitals[i])

        df_neutral=rp_climatology(period_names[1],pathOut,capitals[i])
        df_elnino=rp_climatology(period_names[2],pathOut,capitals[i])
        df_lanina=rp_climatology(period_names[0],pathOut,capitals[i])
        plot_return_wind (df_neutral, df_elnino,df_lanina,capitals[i], names[i],pathOut)
    except:
        continue